In [ ]:
!pip install estnltk==1.7.2

In [3]:
from estnltk.wordnet import Wordnet
import random
import re
import json
import html

In [4]:
wn = Wordnet()

def contains_numbers_or_dot(word):
    pattern = r'[0-9.]'
    return bool(re.search(pattern, word))

uniqdict = {}
for i, synset in enumerate(wn):
    if " " in synset.name or "-" in synset.name:
        continue
    splitted = synset.name.split(".")[0]
    try:
        definition = wn[splitted][0].definition
    except IndexError:
        continue
    if definition is None:
        continue
    if contains_numbers_or_dot(splitted):
        continue
    if len(splitted) > 17:
        continue
    uniqdict[splitted] = definition
    

In [5]:
alphabeticDict = {}

def checkCorrectWordConstruct(word):
    if not re.fullmatch(r'[a-zõüöä]+', word):
        return False
    return True

for key in uniqdict:
    if checkCorrectWordConstruct(key):
        alphabetically = ''.join(sorted(set(key)))
        alphabeticDict[alphabetically] = key + "::" + uniqdict[key]


In [6]:
with open('definitions.json', 'w', encoding="utf-8") as file:
    json.dump(alphabeticDict, file)

In [9]:
file_path = "koik_lehed"

def is_valid_sentence(sentence):
    words = sentence.split(' ')
    for word in words:
        if not re.fullmatch(r'[a-zõüöäA-ZÕÜÖÄ]+\-?\s?', word):
            return False
        if re.fullmatch(r'[A-ZÕÜÖÄ\-]\s?', word):
            return False
        uppercase_count = sum(1 for char in word if char.isupper())
        if uppercase_count > 1:
            return False
        if len(word) > 17 or len(word) < 2:
            return False
    return True

def trim_sentence_end(sentence):
    trimmed_sentence = re.sub(r'[^a-zõüöäA-ZÕÜÖÄ]+$', '', sentence)
    return trimmed_sentence

checked_sentences = []

with open(file_path, 'r', encoding='iso-8859-1') as file:
    for line in file:
        parts = line.split('     ', 1)
        if len(parts) == 1:
            parts = line.split('    ', 1)
        if len(parts) == 1:
            parts = line.split('   ', 1)
        if len(parts) == 2:
            identifier, content = parts
            try:
                decoded_content = html.unescape(content)
                
                sentence = decoded_content.strip()
                sentence = trim_sentence_end(sentence)
                    
                words = sentence.split()
                
                if len(words) >= 4 and len(words) <= 8:
                    if is_valid_sentence(sentence):
                        checked_sentences.append(sentence)
                    else:
                        continue
                    
            except UnicodeDecodeError as e:
                print("UnicodeDecodeError:", e)
        else:
            print("Invalid line format:", line)

In [10]:
with open("sentences.txt", "w", encoding="utf-8") as file:
    for sent in checked_sentences:
        file.write(f"{sent}\n")

In [11]:
print(len(alphabeticDict))
print(len(checked_sentences))

44559
57633
